In [ ]:
from exprel.dataset.semeval_dataset import SemevalDataset
from dotenv import load_dotenv 
load_dotenv()

In [ ]:
data = SemevalDataset("/home/kovacs/projects/exp-relation-extraction/data/semeval_train.txt")

In [ ]:
df = data.to_dataframe()

In [ ]:
len(df)

In [ ]:
df

In [ ]:
from exprel.feature_extractor.extract import FeatureExtractor

extractor = FeatureExtractor()

In [ ]:
import pickle
PIK = "pickle.dat"

with open(PIK, "rb") as f:
    graphs = pickle.load(f)

In [ ]:
one_versus_rest = data.one_versus_rest(df, "Entity-Destination(e1,e2)")

In [ ]:
#one_versus_rest = one_versus_rest.sample(frac=0.2, random_state=1234).sample(frac=1.0, random_state=1234)

In [ ]:
one_versus_rest.iloc[0].sentence

In [ ]:
import pandas as pd

ids = pd.to_numeric(one_versus_rest.sen_id).tolist()
sentences = one_versus_rest.sentence.tolist()
labels = one_versus_rest.one_versus_rest.tolist()
postprocessed_graphs = one_versus_rest.graph.tolist()

In [ ]:
labels.count(1)

In [ ]:
labels.count(0)

In [ ]:
"""
from tqdm import tqdm
graphs = []

for sen in tqdm(sentences):
    output, root = extractor.extract(sen)
    graphs.append(output)
    
import pickle
PIK = "pickle.dat"

with open(PIK, "wb") as f:
    pickle.dump(graphs, f)
"""

In [ ]:
data.set_graphs(graphs)

In [ ]:
df = data.to_dataframe()
one_versus_rest = data.one_versus_rest(df, "Entity-Destination(e1,e2)")

In [ ]:
one_versus_rest

In [ ]:
from exprel.models.model import GraphModel
model = GraphModel()

In [ ]:
from tqdm import tqdm

for id, graph, label in tqdm(zip(ids, postprocessed_graphs, labels)):
    model.featurize_sen_graph(id, graph, label, 3)

In [ ]:
model.select_n_best(7000)

In [ ]:
X, Y = model.get_x_y(one_versus_rest.one_versus_rest.tolist())

In [ ]:
len(X[0])

In [ ]:
from sklearn.model_selection import train_test_split as split

tr_data,tst_data,tr_labels,tst_labels = split(X,Y, test_size=0.3, random_state=1234)

In [ ]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=0).fit(tr_data, tr_labels)

In [ ]:
from sklearn.metrics import precision_recall_fscore_support

lr_pred = clf.predict(tst_data)
for pcf in precision_recall_fscore_support(tst_labels, lr_pred, average=None):
    print(pcf[1])

In [ ]:
list(tr_labels).count(1)

In [ ]:
print("hello")

In [ ]:
list(tst_labels).count(1)

In [ ]:
list(lr_pred).count(1)

In [45]:
import eli5

In [46]:
weights_df = eli5.explain_weights_df(clf)

In [47]:
weights_df

,target,feature,weight
0,1.0,x42,3.554073
1,1.0,x19,2.829777
2,1.0,x1992,2.306070
3,1.0,x13,2.115405
4,1.0,x97,1.625335
...,...,...,...
6357,1.0,x963,-1.105894
6358,1.0,x17,-1.162078
6359,1.0,x376,-1.169265
6360,1.0,x2,-1.343590


In [48]:
feature_graphs = model.get_feature_graphs()

In [51]:
feature_graph_strings = model.get_feature_graph_strings()
most_important_weights = weights_df.iloc[:10].feature.str.strip("x").tolist()
g = feature_graph_strings[model.inverse_relabel[int(most_important_weights[0])]]
print(g)

(u_29 / to  :2 (u_4 / entity2))


In [ ]:
import networkx as nx
all_features = nx.MultiDiGraph()

In [ ]:
for i, t in enumerate(weights_df.iloc[:10].feature.str.strip("x").tolist()):
    node_to_string = {}
    g = feature_graphs[model.inverse_relabel[int(t)]]
    nodes = []
    for n in g.nodes(data=True):
        n_post = n[1]["name"]+"_feature"+str(i)
        nodes.append(n_post)
        node_to_string[n[0]] = n_post
    all_features.add_nodes_from(nodes)
    
    for e in g.edges(data=True):
        all_features.add_edge(node_to_string[e[0]], node_to_string[e[1]], color=e[2]["color"])
        

In [ ]:
from graphviz import Digraph
from graphviz import Source
# Create Digraph object
dot = to_dot(all_features, integ=True)
Source(dot)

In [ ]:
from graphviz import Digraph
from graphviz import Source
# Create Digraph object
dot = to_dot(feature_graphs[model.inverse_relabel[42]])

In [ ]:
dots = []

for t in weights_df.iloc[:10].feature.str.strip("x").tolist():
    dots.append(feature_graphs[model.inverse_relabel[int(t)]])

Source(to_dots(dots))

In [ ]:
import re

def d_clean(string):
    s = string
    for c in '\\=@-,\'".!:;<>/{}[]()#^?':
        s = s.replace(c, '_')
    s = s.replace('$', '_dollars')
    s = s.replace('%', '_percent')
    s = s.replace('|', ' ')
    s = s.replace('*', ' ')
    if s == '#':
        s = '_number'
    keywords = ("graph", "node", "strict", "edge")
    if re.match('^[0-9]', s) or s in keywords:
        s = "X" + s
    return s

def to_dots(graphs, marked_nodes=set(), integ=False):
    lines = [u'digraph finite_state_machine {', '\tdpi=70;']
    # lines.append('\tordering=out;')
    # sorting everything to make the process deterministic
    for i, graph in enumerate(graphs):
        s = "subgraph cluster_" + chr(ord('@')+i+1) + " {"
        node_lines = []

        node_lines.append(s)
        node_to_name = {}
        for node, n_data in graph.nodes(data=True):
            if integ:
                d_node = d_clean(str(node))
            else:    
                d_node = d_clean(n_data["name"])
            printname = d_node
            node_to_name[node] = printname
            if 'expanded' in n_data and n_data['expanded'] and printname in marked_nodes:
                node_line = u'\t{0} [shape = circle, label = "{1}", \
                        style=filled, fillcolor=purple];'.format(
                    d_node, printname).replace('-', '_')
            elif 'expanded' in n_data and n_data['expanded']:
                node_line = u'\t{0} [shape = circle, label = "{1}", \
                        style="filled"];'.format(
                    d_node, printname).replace('-', '_')
            elif 'fourlang' in n_data and n_data['fourlang']:
                node_line = u'\t{0} [shape = circle, label = "{1}", \
                        style="filled", fillcolor=red];'.format(
                    d_node, printname).replace('-', '_')
            elif 'substituted' in n_data and n_data['substituted']:
                node_line = u'\t{0} [shape = circle, label = "{1}", \
                        style="filled"];'.format(
                    d_node, printname).replace('-', '_')
            elif printname in marked_nodes:
                node_line = u'\t{0} [shape = circle, label = "{1}", style=filled, fillcolor=lightblue];'.format(
                    d_node, printname).replace('-', '_')
            else:
                node_line = u'\t{0} [shape = circle, label = "{1}"];'.format(
                    d_node, printname).replace('-', '_')
            node_lines.append(node_line)
        lines += sorted(node_lines)

        edge_lines = []
        for u, v, edata in graph.edges(data=True):
            if 'color' in edata:
                d_node1 = node_to_name[u]
                d_node2 = node_to_name[v]
                edge_lines.append(
                    u'\t{0} -> {1} [ label = "{2}" ];'.format(d_node1, d_node2, edata['color']))

        lines += sorted(edge_lines)
        lines.append('}')
    lines.append('}')
    return u'\n'.join(lines)

def to_dot(graph, marked_nodes=set(), integ=False):
    lines = [u'digraph finite_state_machine {', '\tdpi=70;']
    # lines.append('\tordering=out;')
    # sorting everything to make the process deterministic
    node_lines = []
    node_to_name = {}
    for node, n_data in graph.nodes(data=True):
        if integ:
            d_node = d_clean(str(node))
        else:    
            d_node = d_clean(n_data["name"])
        printname = d_node
        node_to_name[node] = printname
        if 'expanded' in n_data and n_data['expanded'] and printname in marked_nodes:
            node_line = u'\t{0} [shape = circle, label = "{1}", \
                    style=filled, fillcolor=purple];'.format(
                d_node, printname).replace('-', '_')
        elif 'expanded' in n_data and n_data['expanded']:
            node_line = u'\t{0} [shape = circle, label = "{1}", \
                    style="filled"];'.format(
                d_node, printname).replace('-', '_')
        elif 'fourlang' in n_data and n_data['fourlang']:
            node_line = u'\t{0} [shape = circle, label = "{1}", \
                    style="filled", fillcolor=red];'.format(
                d_node, printname).replace('-', '_')
        elif 'substituted' in n_data and n_data['substituted']:
            node_line = u'\t{0} [shape = circle, label = "{1}", \
                    style="filled"];'.format(
                d_node, printname).replace('-', '_')
        elif printname in marked_nodes:
            node_line = u'\t{0} [shape = circle, label = "{1}", style=filled, fillcolor=lightblue];'.format(
                d_node, printname).replace('-', '_')
        else:
            node_line = u'\t{0} [shape = circle, label = "{1}"];'.format(
                d_node, printname).replace('-', '_')
        node_lines.append(node_line)
    lines += sorted(node_lines)

    edge_lines = []
    for u, v, edata in graph.edges(data=True):
        if 'color' in edata:
            d_node1 = node_to_name[u]
            d_node2 = node_to_name[v]
            edge_lines.append(
                u'\t{0} -> {1} [ label = "{2}" ];'.format(d_node1, d_node2, edata['color']))

    lines += sorted(edge_lines)
    lines.append('}')
    return u'\n'.join(lines)

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [ ]:
chi2_selector = SelectKBest(chi2, k=10)
X_kbest = chi2_selector.fit_transform(X, Y)

In [ ]:
chi2_selector.get_support(indices=True)

In [ ]:
for i, g in enumerate(feature_graphs):
    if len(g.edges()) != 1:
        print(len(g.edges()))

In [ ]:
dot = to_dot(feature_graphs[2450])
Source(dot)

In [ ]:
dot = to_dot(graphs[0])
Source(dot)

In [ ]:
tests = []

for l in model.lexgraphs.gen_lex_subgraphs(graphs[0], 1):
    print(l)
    tests.append(l[1])

In [ ]:
dot = to_dot(tests[3])
Source(dot)

In [ ]:
from networkx.convert import from_dict_of_dicts as fdd
from networkx.convert import to_dict_of_dicts as tdd

H_dict = tdd(graphs[0])
H_dict

In [ ]:
import networkx as nx
G = nx.MultiDiGraph()

fdd(H_dict,create_using=nx.MultiDiGraph()).edges(data=True)

In [ ]:
dot = to_dot(graphs[0], integ=True)
Source(dot)

In [ ]:
def gen_subgraphs(M, no_edges):
    #print(no_edges)
    """M must be dict of dicts, see networkx.convert.to_dict_of_dicts.
    Generates dicts of dicts, use networkx.convert.from_dict_of_dicts"""
    if no_edges == 0:
        yield from ({v: {}} for v in M)
        return
    for s_graph in gen_subgraphs(M, no_edges-1):
        if no_edges >= 2:
            yield s_graph
        # print('sgraph:', s_graph)
        for node in M:
            #print(node)
            #print(M[node].items())
            for neighbor, edge in M[node].items():
                #print(neighbor)
                #print(edge)
                if node in s_graph and neighbor in s_graph[node]:
                    continue
                if node not in s_graph and neighbor not in s_graph:
                    continue

                new_graph = s_graph.copy()
                if node not in new_graph:
                    new_graph[node] = {neighbor: edge}
                else:
                    new_graph[node][neighbor] = edge
                yield new_graph

In [ ]:
for i in gen_subgraphs(H_dict, 1):
    #pass
    print(i)
    sgraph = fdd(i, create_using=nx.MultiDiGraph())
    print(sgraph.edges(data=True))

In [ ]:
t = graphs[0].copy()

In [ ]:
from collections import Counter

s = Counter()

s["banana"] += 1

In [ ]:
s.most_common(1)